In [1]:
import sys
sys.path.append('../')

In [2]:
from pathlib import Path
import yaml

import pandas as pd

from legacy_code_assistant.knowledge_base.knowledge_builder import KnowledgeBaseBuilder
from legacy_code_assistant.knowledge_base.knowledge_builder import CodeAnalyzer

In [3]:
with open('credentials.yaml', "r") as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

In [4]:
path = Path() / '..' / '..' / 'Django-School-Management-System'
# path = Path() / '..' / 'test_code'


In [5]:
paths = list(path.rglob('**/*.py'))
ca = CodeAnalyzer(paths)
results = ca.analyze()
print(len(results))
results

218


[{'docstring': None,
  'code': 'def main():\n    os.environ.setdefault(\'DJANGO_SETTINGS_MODULE\', \'school_app.settings\')\n    try:\n        from django.core.management import execute_from_command_line\n    except ImportError as exc:\n        raise ImportError(\n            "Couldn\'t import Django. Are you sure it\'s installed and available on your PYTHONPATH environment variable? Did you forget to activate a virtual environment?"\n            ) from exc\n    execute_from_command_line(sys.argv)\n',
  'code_start_line': 7,
  'code_end_line': 17,
  'file': '../../Django-School-Management-System/manage.py',
  'module': 'manage',
  'name': 'main',
  'parent': 'manage',
  'type': 'function'},
 {'docstring': "Django's command-line utility for administrative tasks.",
  'code': '"""Django\'s command-line utility for administrative tasks."""\nimport os\nimport sys\n\n\ndef main():\n    os.environ.setdefault(\'DJANGO_SETTINGS_MODULE\', \'school_app.settings\')\n    try:\n        from django.c

In [6]:
df = pd.DataFrame(results)
df['len'] = df['code'].apply(lambda x: len(x))
df = df.sort_values('len', ascending=False)
df.head(10)

,docstring,code,code_start_line,code_end_line,file,module,name,parent,type,len
169,None,from django.contrib import messages\nfrom djan...,0,NaN,../../Django-School-Management-System/apps/cor...,views,views,NaN,module,9495
103,None,from django.contrib.auth.decorators import log...,0,NaN,../../Django-School-Management-System/apps/fin...,views,views,NaN,module,4093
33,None,from django.contrib import messages\nfrom djan...,0,NaN,../../Django-School-Management-System/apps/res...,views,views,NaN,module,3621
214,Django settings for school_app project.\n\nGen...,"""""""\nDjango settings for school_app project.\n...",0,NaN,../../Django-School-Management-System/school_a...,settings,settings,NaN,module,3281
62,None,import csv\nfrom django.contrib.auth.mixins im...,0,NaN,../../Django-School-Management-System/apps/stu...,views,views,NaN,module,3081
73,None,import csv\nimport os\nfrom io import StringIO...,0,NaN,../../Django-School-Management-System/apps/stu...,signals,signals,NaN,module,2715
123,None,from django.db import models\nfrom django.urls...,0,NaN,../../Django-School-Management-System/apps/fin...,models,models,NaN,module,2081
126,None,"from django.db import migrations, models\nimpo...",0,NaN,../../Django-School-Management-System/apps/fin...,0001_initial,0001_initial,NaN,module,2012
69,None,"@receiver(post_save, sender=StudentBulkUpload)...",14,68.0,../../Django-School-Management-System/apps/stu...,signals,create_bulk_student,signals,function,2003
84,None,"from django.db import migrations, models\nimpo...",0,NaN,../../Django-School-Management-System/apps/stu...,0001_initial,0001_initial,NaN,module,1911


In [7]:
len(df.loc[df['type'].isin(['function', 'method', 'class'])])

151

In [8]:
df.to_csv('knowledge_base.csv', index=False)

<h1>Generating docstrings / descriptions</h2>

In [9]:
from legacy_code_assistant.knowledge_base.description_generator import CodeConditionedGenerator

In [10]:
generator = CodeConditionedGenerator(credentials_path='credentials.yaml', data_path='knowledge_base.csv')
generator.generate_docstrings()

  2%|▏         | 3/151 [01:26<1:11:08, 28.84s/it]


KeyboardInterrupt: 

<h1>Make vectorstore based on code</h1>

In [ ]:
from legacy_code_assistant.knowledge_base.knowledge_builder import KnowledgeBaseBuilder

In [32]:
kbb_code = KnowledgeBaseBuilder(index_name='code_based_index')
kbb_code.initialize_faiss_based_on_df(df, text_column='code')

kbb_code.search('change students results from their exams', k=10)

No sentence-transformers model found with name /home/laszer/.cache/torch/sentence_transformers/microsoft_codebert-base. Creating a new one with MEAN pooling.


[(Document(page_content="class Meta:\n    ordering = ['subject']", metadata={'docstring': nan, 'code_start_line': 24, 'code_end_line': 25.0, 'file': '../../Django-School-Management-System/apps/result/models.py', 'module': 'models', 'name': 'Meta', 'parent': nan, 'type': 'class', 'len': 39, 'generated_docstring': "The `Meta` class is used to specify options that don't necessarily relate directly to a single field in a model or form. \n\nAttributes:\n    ordering (list): A list of strings that define the default ordering of objects when they are retrieved from the database. \n                     In this case, objects will be ordered by the 'subject' field."}),
  30.11378),
 (Document(page_content="class Meta:\n    ordering = ['student', 'term']", metadata={'docstring': nan, 'code_start_line': 21, 'code_end_line': 22.0, 'file': '../../Django-School-Management-System/apps/finance/models.py', 'module': 'models', 'name': 'Meta', 'parent': nan, 'type': 'class', 'len': 47, 'generated_docstrin

In [ ]:
kbb_code.save_index()

<h1>Make vectorstore based on generated docstrings</h1>

In [27]:
df = pd.read_csv('functions_with_generated_docstrings.csv')

In [28]:
from langchain.embeddings import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=credentials['Deployment_embeddings'],
    openai_api_version="2023-05-15",
)

In [29]:
embeddings.model

'text-embedding-ada-002'

In [30]:
kbb_docs = KnowledgeBaseBuilder(index_name='docstring_based_index', model=embeddings)
kbb_docs.initialize_faiss_based_on_df(df, text_column='generated_docstring')

In [31]:
kbb_docs.search('definition of student object with results of their exams', k=10)[0][0]

Document(page_content="This view generates a summary of students' test and exam scores for the current session and term.\n\nUpon a GET request, it filters the Result model for objects that match the current\nsession and term from the request context. It then aggregates the scores for each student,\ngrouping their subjects and calculating the total test and exam scores.\n\nThe aggregated data includes:\n    - The list of subjects per student\n    - Total test score across all subjects for each student\n    - Total exam score across all subjects for each student\n    - The combined total of test and exam scores for each student\n\nThe view then constructs a context dictionary with this aggregated data under the key\n'results', which it passes to the 'result/all_results.html' template for rendering.\n\nThis view requires the user to be logged in, as it inherits from LoginRequiredMixin.", metadata={'docstring': nan, 'code': "class ResultListView(LoginRequiredMixin, View):\n\n    def get(se

<h1>RAG</h1>

In [ ]:
kbb.save_index()